In [1]:
!pip install deepface
!pip install sklearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 5.3 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116932 sha256=bd430c3edea85498fe15df49e37eaeb7fb5cb6aaf48a97ebcc118a3fe48d9446
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post9-py3-none-any.whl size=2952 sha256=41153d01cf61233283f52ca7a5aa928d5ae3eab701fdee1f4099e4200f3999e3
  Stored in directory: /root/.cache/pip/wheels/33/a3/d2/092b519e9522b4c91608b7dcec0dd9051fa1bff4c45f4502d1
Successfully built sklearn


In [2]:
import tarfile
tar = tarfile.open("/kaggle/input/lfwpeople/lfw-funneled.tgz")
tar.extractall("/kaggle/working")
tar.close()

In [3]:
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
import pandas as pd
from deepface import DeepFace
import PIL.Image as Image


def get_dataset():
    """
    t[0]: name
    t[1]: database image
    t[2]: same person image | different person name
    if t[2] == different person name that t[3] = different person image
    :return:
    """
    # 按行读取 pairsDevTrain.txt
    test_dataset = []
    with open("/kaggle/input/lfwpeople/pairsDevTest.txt", "r") as f:
        lines = f.readlines()
        lines.pop(0)
        for line in lines:
            t = line.split("\t")
            t[-1] = t[-1].removesuffix("\n")
            test_dataset.append(t)
    return test_dataset


def verify_test_dataset(dataset, model_name="VGG-Face", distance_metric="cosine",
                        detector_backend="opencv"):
    actuals = []
    predictions = []
    predictions_halftone = []
    if not os.path.exists(f"/kaggle/working/{model_name}_{distance_metric}_{detector_backend}.csv"):
        with open(f"/kaggle/working/{model_name}_{distance_metric}_{detector_backend}.csv", "w") as f:
            f.write("1st person,2nd person,dist,verified,dist_halftone,verified_halftone\n")

    for img in dataset:
        same = True if len(img) == 3 else False
        first_img = f"/kaggle/input/label-face-in-wild/lfw/{img[0]}/{img[0]}_{int(img[1]):04d}.jpg"
        second_img = f"/kaggle/input/label-face-in-wild/lfw/{img[0]}/{img[0]}_{int(img[2]):04d}.jpg" \
            if same else f"/kaggle/input/label-face-in-wild/lfw/{img[2]}/{img[2]}_{int(img[3]):04d}.jpg"
        
        second_img_halftone = np.array(Image.open(second_img).convert("1").convert("RGB"))
        first_img = np.array(Image.open(first_img))
        second_img = np.array(Image.open(second_img))
        # opencv expects bgr instead of rgb
        first_img = first_img[:, :, ::-1]
        second_img = second_img[:, :, ::-1]
        second_img_halftone = second_img_halftone[:, :, ::-1]
        
        result = DeepFace.verify(first_img, second_img, model_name=model_name, distance_metric=distance_metric,
                                 detector_backend=detector_backend, enforce_detection=False)
        result_halftone = DeepFace.verify(first_img, second_img_halftone, model_name=model_name,
                                          distance_metric=distance_metric,
                                          detector_backend=detector_backend, enforce_detection=False)

        data = pd.DataFrame({"1st person": [img[0]],
                             "2nd person": [img[0] if same else img[2]],
                             "dist": [result["distance"]],
                             "verified": [result["verified"]],
                             "dist_halftone": [result_halftone["distance"]],
                             "verified_halftone": [result_halftone["verified"]]
                             })

        data.to_csv(f"./{model_name}_{distance_metric}_{detector_backend}.csv", mode="a", header=False, index=False)

        print(f"{img} dist: {result['distance']} verified: {result['verified']} dist_halftone: {result_halftone['distance']} verified_halftone: {result_halftone['verified']}")
        actuals.append(True if same else False)
        predictions.append(result["verified"])
        predictions_halftone.append(result_halftone["verified"])

    accuracy = 100 * accuracy_score(actuals, predictions)
    precision = 100 * precision_score(actuals, predictions)
    recall = 100 * recall_score(actuals, predictions)
    f1 = 100 * f1_score(actuals, predictions)
    tn, fp, fn, tp = confusion_matrix(actuals, predictions).ravel()

    metric = pd.DataFrame({"accuracy": [accuracy],
                           "precision": [precision],
                           "recall": [recall],
                           "f1": [f1],
                           "tn": [tn],
                           "fp": [fp],
                           "fn": [fn],
                           "tp": [tp]
                           })
    metric.to_csv(f"/kaggle/working/{model_name}_{distance_metric}_{detector_backend}.csv", mode="a", header=True, index=False)

    accuracy_halftone = 100 * accuracy_score(actuals, predictions_halftone)
    precision_halftone = 100 * precision_score(actuals, predictions_halftone)
    recall_halftone = 100 * recall_score(actuals, predictions_halftone)
    f1_halftone = 100 * f1_score(actuals, predictions_halftone)
    tn_halftone, fp_halftone, fn_halftone, tp_halftone = confusion_matrix(actuals, predictions_halftone).ravel()
    metric_halftone = pd.DataFrame({"h_accuracy": [accuracy_halftone],
                                    "h_precision": [precision_halftone],
                                    "h_recall": [recall_halftone],
                                    "h_f1": [f1_halftone],
                                    "h_tn": [tn_halftone],
                                    "h_fp": [fp_halftone],
                                    "h_fn": [fn_halftone],
                                    "h_tp": [tp_halftone]
                                    })
    metric_halftone.to_csv(f"/kaggle/working/{model_name}_{distance_metric}_{detector_backend}.csv", mode="a", header=True,
                           index=False)


# Euclidean L2 form seems to be more stable than cosine and regular Euclidean distance based on experiments.

# Face recognition models are actually CNN models and they expect standard sized inputs. So, resizing is required before
# representation. To avoid deformation, deepface adds black padding pixels according to the target size argument
# after detection and alignment.

# RetinaFace and MTCNN seem to over-perform in detection and alignment stages, but they
# are much slower. If the speed of your pipeline is more important, then you should use opencv or ssd. On the other
# hand, if you consider the accuracy, then you should use retinaface or mtcnn.
if __name__ == '__main__':
    models = [
        "VGG-Face",
        "Facenet",
        "Facenet512",
        "OpenFace",
        "DeepFace",
        "DeepID",
        "ArcFace",
        "Dlib",
        "SFace",
    ]
    backends = [
        'opencv',
        'ssd',
        'dlib',
        'mtcnn',
        'retinaface',
        'mediapipe',
        'yolov8',
        'yunet',
    ]
    metrics = ["cosine", "euclidean", "euclidean_l2"]
    dataset = get_dataset()
    # t = dataset[0]
    # res = df.verify(f"./lfw_funneled/{t[0]}/{t[0]}_{int(t[1]):04d}.jpg",
    #                 f"./lfw_funneled/{t[0]}/{t[0]}_{int(t[2]):04d}.jpg",
    #                 model_name="VGG-Face", distance_metric="cosine", detector_backend="opencv")
    # print(res)
    # facenet proved to not be a good model for halftone face recognition
    verify_test_dataset(dataset, model_name=models[0], distance_metric=metrics[1], detector_backend=backends[0])


Directory  /root /.deepface created
Directory  /root /.deepface/weights created
vgg_face_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5
To: /root/.deepface/weights/vgg_face_weights.h5
100%|██████████| 580M/580M [00:01<00:00, 345MB/s]


['Abdullah_Gul', '13', '14'] dist: 0.28609922423450596 verified: True dist_halftone: 0.5131605209078882 verified_halftone: True
['Abdullah_Gul', '13', '16'] dist: 0.4345321888148093 verified: True dist_halftone: 0.5546657021248516 verified_halftone: True
['Abdullatif_Sener', '1', '2'] dist: 0.3067683281298301 verified: True dist_halftone: 0.38523682870757747 verified_halftone: True
['Adel_Al-Jubeir', '1', '3'] dist: 0.2499771257449424 verified: True dist_halftone: 0.36364087688287594 verified_halftone: True
['Al_Pacino', '1', '2'] dist: 0.32166355863823853 verified: True dist_halftone: 0.6172695968599591 verified_halftone: False
['Alan_Greenspan', '1', '5'] dist: 0.2777969849081003 verified: True dist_halftone: 0.46188376755087374 verified_halftone: True
['Albert_Costa', '2', '6'] dist: 0.44441636124867334 verified: True dist_halftone: 0.598045648285905 verified_halftone: True
['Albert_Costa', '4', '6'] dist: 0.4787032721470334 verified: True dist_halftone: 0.607291684108223 verified_h